22) Top 5 de lenguajes que son usados por usuarios bilingües. (⭐⭐).

**Analisis previo**

1) Consideramos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o igual a 1.

2) Consideramos aquellos usuarios que SOLO saben 2 idiomas.

3) Tenemos valores "nan" en la columna "babel_lang" la cual nos interesa, entonces antes de trabajar eliminamos estos datos.

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

from google.colab import drive
drive.mount("/content/drive")

### **Resolucion explicativa**

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion de Datos/Colab Notebooks/TP1/languages.csv', header=True, inferSchema=True)
rdd = df.rdd

Pasos

1) Con la transformacion map() buscamos obtener registros de la forma (id de usuario, codigo de idioma, nivel) y mapear el nivel N a un valor numerico.

2) Con la transformacion filter() buscamos quedarnos con los usuarios que saben un idioma considerando que un usuario sabe un idioma cuando tiene un nivel de babel mayor o igual a 1.

3) Con la transformacion map() buscamos obtener registros de la forma (id de usuario, codigo de idioma) para descartar el nivel ya que solo lo necesitabamos para filtrar aquellos usuarios que nos interesaban.

In [3]:
rdd_idiomas = rdd.filter(lambda x: x.babel_lang != "nan")\
                 .map(lambda x: (x.babel_user, x.babel_lang, 4.5) if x.babel_level == "N" else (x.babel_user, x.babel_lang, int(x.babel_level))).filter(lambda x: x[2] > 0)\
                 .map(lambda x: (x[0], x[1])).cache()

Pasos

4) Con la transformacion map() buscamos obtener registros de la forma (id de usuario, 1) para poder contar cuantos idiomas saben hablar.

5) Con la transformacion reduceByKey() buscamos sumar los registros para una misma clave.

6) Con la transformacion filter() buscamos quedarnos con aquellos usuarios que SOLO sean bilingues.

In [4]:
rdd_usuarios_bilingues = rdd_idiomas.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] == 2)

Pasos

7) Con la transformacion leftOuterJoin() buscamos obtener un RDD de la forma (usuario, (2, codigo de idioma)).

In [5]:
rdd_idiomas_bilingues = rdd_usuarios_bilingues.leftOuterJoin(rdd_idiomas)

Pasos

8) Con la transformacion map() buscamos obtener registros de la forma (codigo de idioma, 1) para poder contarlos.

9) Con la transformacion reduceByKey() buscamos sumar los registros para una misma clave.

10) Con la accion takeOrdered() buscamos hallar los primeros 5 registros de mayor a menor.

In [6]:
resultado = rdd_idiomas_bilingues.map(lambda x: (x[1][1], 1)).reduceByKey(lambda x,y: x+y).takeOrdered(5, lambda x: -x[1])

### **Resolucion reducida**

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion de Datos/Colab Notebooks/TP1/languages.csv', header=True, inferSchema=True)
rdd = df.rdd

rdd_idiomas = rdd.filter(lambda x: x.babel_lang != "nan")\
                 .map(lambda x: (x.babel_user, x.babel_lang, 4.5) if x.babel_level == "N" else (x.babel_user, x.babel_lang, int(x.babel_level))).filter(lambda x: x[2] > 0)\
                 .map(lambda x: (x[0], x[1])).cache()

rdd_usuarios_bilingues = rdd_idiomas.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] == 2)

rdd_idiomas_bilingues = rdd_usuarios_bilingues.leftOuterJoin(rdd_idiomas)

resultado = rdd_idiomas_bilingues.map(lambda x: (x[1][1], 1)).reduceByKey(lambda x,y: x+y).takeOrdered(5, lambda x: -x[1])

### **Respuesta**

In [32]:
resultado

[('es', 3112), ('en', 3091), ('fr', 73), ('ca', 56), ('de', 29)]